## Описание проекта

### 🎯 Задача

Обучить модель классифицировать комментарии как **токсичные** или **нетоксичные**.  
В распоряжении — набор размеченных данных.

> 📌 Целевая метрика: **F1-score**  
> 🎯 Требование: значение **F1** должно быть **не меньше 0.75**

**Исходные данные**  
Файл: `toxic_comments.csv`  
- `text` — текст комментария  
- `toxic` — целевой признак (1 — токсичный, 0 — нетоксичный)

---

### 🧭 План исследования

Будут протестированы 3 подхода:

1. **Spacy (лемматизация) + TF-IDF + LogisticRegression**
2. **BERT (базовая модель) + LogisticRegression**
3. **BERT (после файнтюна)**

> ⚙️ Модель №3 обучалась на арендованном сервере:  
> GPU: `1×V100 32GB`, CPU: `20 vCPU`, RAM: `64 GB`, SSD: `400 GB`

---

### 📚 Содержание

#### Импорт библиотек

#### Подготовка
- 2.1 Загрузка данных  
- 2.2 Подготовка датасета для TF-IDF  
  - 2.2.1 Spacy  
  - 2.2.2 NLTK  
  - 2.2.3 Пайплайн  
- 2.3 Подготовка BERT (базовая модель)

#### Обучение
- 3.1 TF-IDF  
- 3.2 BERT (базовая модель)  
  - 3.2.1 Код эмбединга моделью BERT  
- 3.3 BERT (после файнтюна)  
  - 3.3.1 Код файнтюна модели BERT  
  - 3.3.2 Код инференса

#### Выводы

---

### 🧾 Итоговый вывод

В ходе проекта были рассмотрены три подхода к классификации токсичных комментариев:

- **Spacy + TF-IDF + LogisticRegression** — простая и быстрая, но недостаточно точная модель.
- **BERT (базовая модель) + LogisticRegression** — улучшение качества, но без дообучения ограничены возможности.
- **BERT (дообученный)** — достигнуто требуемое значение метрики F1 = **0.82**, что превышает порог 0.75.

Дообучение модели BERT оказалось наиболее эффективным решением, обеспечив качество, приемлемое для заказчика. Однако стоит учитывать высокие вычислительные затраты и время обучения (около 4.5 часов на мощном сервере).

Таким образом, баланс между качеством и ресурсами остаётся важным фактором при выборе подхода, и в текущем случае преимущества дообученной модели BERT перевешивают её недостатки.

---

**Рекомендация:** Для дальнейших проектов стоит рассмотреть упрощённые модели или оптимизацию процесса дообучения, чтобы снизить ресурсоёмкость без значительной потери качества.


In [1]:
!pip install gdown -q

## Импорт бибилиотек

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import re
import spacy
import torch 
import transformers
import time
import gdown

from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    f1_score,
    recall_score, 
    precision_score,
    confusion_matrix
)

seed = 42

## Подготовка

### Загрузка данных

Загружаем 3 варианта данных под каждый вариант:
1. сырой датасет (df) для варианта TF-IDF
2. датасет с эмбедингами базовой моделью BERT (bert_basic_features)
3. предсказания на тестовой выборке моделью BERT finetuned (bert_finetuned_prediction)

In [3]:
# 1. сырой датасет
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
print("✅ Сырой датасет загружен:")
display(df.head())


# 2. предсказания берта на валидации после файнтюна
csv_url = "https://drive.google.com/uc?export=download&id=1NUbZWztWcmzge784nic5y5I6vwg9aczi"
bert_finetuned_prediction_val = pd.read_csv(csv_url)
print("✅ Предсказания BERT валидация загружены:")
display(bert_finetuned_prediction_val.head())


# 3. предсказания берта на тесте после файнтюна
csv_url = "https://drive.google.com/uc?export=download&id=18zaoWg5QeXOgAc7yA9Y8Lb3xDu4Sw4az"
bert_finetuned_prediction_test = pd.read_csv(csv_url)
print("✅ Предсказания BERT тест загружены:")
display(bert_finetuned_prediction_test.head())

# 4. принаки после эмбединга базовой моделью берт
npy_file_id = "1sOAFvqh25epbD2hygxm4YgXpIX3cM14-"
npy_url = f"https://drive.google.com/uc?id={npy_file_id}"
npy_filename = "bert_features.npy"

# Скачиваем корректно через gdown
gdown.download(npy_url, npy_filename, quiet=False)

bert_basic_features = np.load(npy_filename, allow_pickle=True)
print("✅ Признаки эмбединги загружены:")
print(bert_basic_features.shape)

corpus = df['text']
y = df['toxic']
bert_corpus = pd.DataFrame(bert_basic_features)

✅ Сырой датасет загружен:


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


✅ Предсказания BERT валидация загружены:


,text,true_label,predicted_label,logit_class_0,logit_class_1
0,He can go fuck himself sideways with a spiky s...,1,1,-1.777386,1.081903
1,However: if what you mean is that you want to ...,0,0,2.607142,-1.885007
2,Thanks for the welcome!\nHello Cobaltbluetony....,0,0,2.607144,-1.885006
3,"Too many citations \n\nDuncan, I trimmed some ...",0,0,2.607143,-1.885007
4,"No it is not reasonable to assume. In fact, vi...",0,0,2.607142,-1.885007


✅ Предсказания BERT тест загружены:


,text,true_label,predicted_label,logit_class_0,logit_class_1
0,"""\n Ok, an seems pretty clear by now what edit...",0,0,2.607143,-1.885006
1,Proposal for standard infobox for History of [...,0,0,2.607144,-1.885006
2,"""\n\n Hi, Pompous Ass! ;) \n\nI know you thin...",1,1,-1.776320,1.082370
3,I am under attack! \n\nYou guys are censoring ...,0,1,-1.728215,1.096051
4,Why include Peter and Emerich?==\n\nPeter the ...,0,0,2.607143,-1.885006


Downloading...
From (original): https://drive.google.com/uc?id=1sOAFvqh25epbD2hygxm4YgXpIX3cM14-
From (redirected): https://drive.google.com/uc?id=1sOAFvqh25epbD2hygxm4YgXpIX3cM14-&confirm=t&uuid=6d85bccf-f190-41e2-9f47-4cb1d06ea58f
To: /home/jovyan/work/bert_features.npy
100%|██████████| 489M/489M [00:06<00:00, 75.7MB/s] 


✅ Признаки эмбединги загружены:
(159292, 768)


### Подготовка датасета для TF-IDF

In [4]:
# Датасет с результатами
results = pd.DataFrame(columns=(
    'Lemm method',
    'F1 val',
    'F1 test',
    'Precision val',
    'Precision test',
    'Recall val',
    'Recall test',
    'Training time (s)')
)

#### Spacy

In [7]:
corpus = corpus[:5]
corpus

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object

In [8]:
# Токенизация / лемматизация бибилотекоый Spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner', 'textcat', 'attribute_role'])

def spacy_clean_batch(texts):
    # Оборачиваем генератор в tqdm для отображения прогресса
    for doc in tqdm(nlp.pipe(texts, batch_size=1000, n_process=1), total=len(texts), desc="Lemmatizing"):
        yield ' '.join(
            token.lemma_.lower()
            for token in doc
            if not token.is_stop        # исключаем стоп-слова
            and not token.is_punct      # исключаем пунктуацию
            and not token.like_url      # исключаем ссылки
            and not token.like_email    # исключаем email'ы
            and not token.is_space      # исключаем пробелы
            and not token.is_digit      # исключаем чисто цифровые токены
            and token.is_alpha          # оставляем только буквенные токены
        )

# корпус должен быть списком строк, а не pandas Series
spacy_start = time.time()
lemmatized_corpus = list(spacy_clean_batch(corpus.tolist()))
spacy_lemm_time = time.time() - spacy_start
print(f'Время лемматизации spacy: {spacy_lemm_time}')

Lemmatizing: 100%|██████████| 5/5 [00:00<00:00, 116.06it/s]

Время лемматизации spacy: 0.04584813117980957


In [9]:
# Сохраним результат в файл csv
lc_spacy_out = pd.DataFrame({'lemmatized_text': lemmatized_corpus})
corpus.head()

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: text, dtype: object

In [7]:
# Разделение на train / test
X_trainval_tf_idf_spacy, X_test_tf_idf_spacy, y_trainval_tf_idf_spacy, y_test_tf_idf_spacy  = train_test_split(
    lemmatized_corpus, y, test_size=0.10, stratify=y, random_state=seed
)

#### NLTK

In [10]:
# Загружаем ресурсы
# Скачивает токенизатор Punkt — нужен для разбиения текста на слова (word_tokenize)
nltk.download('punkt')
# Скачивает словарь WordNet — используется для лемматизации WordNetLemmatizer
nltk.download('wordnet')
# Скачивает модель POS-теггера — определяет часть речи для каждого слова (nltk.pos_tag)
nltk.download('averaged_perceptron_tagger')
# Скачивает список английских стоп-слов — используется для фильтрации (stopwords.words('english'))
nltk.download('stopwords')


lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
pattern = re.compile(r'\b[a-zA-Z]{2,}\b')

# Функция сопоставления POS
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # по умолчанию

# Функция лемматизации текста
def fast_lemmatize(texts):
    results = []
    for text in texts:
        tokens = pattern.findall(text.lower())  # только слова
        tagged = pos_tag(tokens)                # POS-теги
        lemmas = [
            lemmatizer.lemmatize(word, get_wordnet_pos(pos))
            for word, pos in tagged
            if word not in stops
        ]
        results.append(' '.join(lemmas))
    return results

# Пример использования (если corpus — Series, делайте .astype(str))
nltk_start = time.time()
lemmatized_corpus_nltk = fast_lemmatize(corpus)
nltk_lemm_time = time.time() - nltk_start
print(f'Время лемматизации nltk: {nltk_lemm_time}')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Время лемматизации nltk: 1.6695582866668701


In [11]:
# Сохраним результат в файл csv
lc_nltk_out = pd.DataFrame({'lemmatized_text': lemmatized_corpus})

In [12]:
lc_nltk_out.head()

,lemmatized_text
0,explanation edit username hardcore metallica f...
1,match background colour seemingly stuck thank ...
2,hey man try edit war guy constantly remove rel...
3,real suggestion improvement wonder section sta...
4,sir hero chance remember page


In [11]:
# Разделение на train / test
X_trainval_tf_idf_nltk, X_test_tf_idf_nltk, y_trainval_tf_idf_nltk, y_test_tf_idf_nltk  = train_test_split(
    lemmatized_corpus_nltk, y, test_size=0.10, stratify=y, random_state=seed
)

#### Пайплайн

In [12]:
# пайплайн (для корректной подготовки данных без утечек)
pipeline_tf_idf = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 1))),
    ('svd', TruncatedSVD(n_components=200, random_state=seed)), 
    ('clf', LogisticRegression(class_weight='balanced', penalty='l1', solver='saga', random_state=seed))
])

param_distributions_tf_idf = {
    'clf': [LogisticRegression(class_weight='balanced', penalty='l1', solver='saga', random_state=seed)]
}

### Подготовка BERT (базовая модель)

In [13]:
# Разделение на train / test
X_trainval_bert_basic, X_test_bert_basic, y_trainval_bert_basic, y_test_bert_basic = train_test_split(
    bert_basic_features, y, test_size=0.10, stratify=y, random_state=seed
)

X_trainval_bert_basic.shape

(143362, 768)

In [14]:
# Параметры для GridSearch
param_distributions_bert_basic = {
    'C': np.arange(.5, 1, .5)
}

## Обучение

In [15]:
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)

In [16]:
scoring = {
    'f1': 'f1',
    'roc_auc': 'roc_auc',
    'precision': 'precision',
    'recall': 'recall'
}

### TF_IDF

In [21]:
# обучение GridSearchCV
def tf_idf_training(name, X_trainval, X_test, y_trainval, y_test):
    global results
    
    start = time.time()
    rs = GridSearchCV(
        pipeline_tf_idf,
        param_grid=param_distributions_tf_idf,
        scoring=scoring,
        cv=cv,
        n_jobs=-1,
        refit='f1'
    )
    
    rs.fit(X_trainval, y_trainval)
    rs_search_time = time.time() - start

    # Извлекаем индекс лучшей модели
    best_idx = rs.best_index_
    cv_results_df = pd.DataFrame(rs.cv_results_)

    # Предсказание
    preds = rs.predict(X_test)
    
    # Собираем метрики в словарь
    row = {
        'Lemm method': name,
        'F1 val': round(rs.best_score_, 2),
        'Precision val': round(cv_results_df.loc[best_idx, 'mean_test_precision'], 2),
        'Recall val': round(cv_results_df.loc[best_idx, 'mean_test_recall'], 2),
        'F1 test': round(f1_score(y_test, preds), 2),
        'Precision test': round(precision_score(y_test, preds), 2),
        'Recall test': round(recall_score(y_test, preds), 2),
        'Training time (s)': round(rs_search_time, 2)
    }

    # Добавляем строку к датафрейсу
    results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

In [22]:
tf_idf_training(
    'NLTK + TF-IDF + Logreg', 
    X_trainval_tf_idf_nltk, 
    X_test_tf_idf_nltk, 
    y_trainval_tf_idf_nltk, 
    y_test_tf_idf_nltk
)

tf_idf_training(
    'SPACY + TF-IDF + Logreg', 
    X_trainval_tf_idf_spacy, 
    X_test_tf_idf_spacy, 
    y_trainval_tf_idf_spacy, 
    y_test_tf_idf_spacy
)

C:\Users\Vladimir\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\Users\Vladimir\anaconda3\envs\practicum\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [23]:
results

,Lemm method,F1 val,F1 test,Precision val,Precision test,Recall val,Recall test,Training time (s)
0,NLTK + TF-IDF + Logreg,0.63,0.64,0.51,0.52,0.84,0.85,224.33
1,SPACY + TF-IDF + Logreg,0.64,0.66,0.52,0.54,0.83,0.85,194.52


### BERT (базовая модель)

In [24]:
# обучение GridSearchCV
start = time.time()

bert_basic_rs = GridSearchCV(
    LogisticRegression(class_weight='balanced', penalty='l1', solver='saga', random_state=seed),
    param_grid=param_distributions_bert_basic,
    scoring=scoring,
    cv=cv,
    n_jobs=-1,
    refit='f1'
)

bert_basic_rs.fit(X_trainval_bert_basic, y_trainval_bert_basic)

bert_basic_rs_search_time = time.time() - start

preds_bert_basic = bert_basic_rs.predict(X_test_bert_basic)

# Извлекаем индекс лучшей модели
best_idx = bert_basic_rs.best_index_
cv_results_df = pd.DataFrame(bert_basic_rs.cv_results_)

# Собираем метрики в строку
row = {
    'Lemm method': 'BERT_embedding + Logreg',
    'F1 val': round(bert_basic_rs.best_score_, 2),
    'Precision val': round(cv_results_df.loc[best_idx, 'mean_test_precision'], 2),
    'Recall val': round(cv_results_df.loc[best_idx, 'mean_test_recall'], 2),
    'F1 test': round(f1_score(y_test_bert_basic, preds_bert_basic), 2),
    'Precision test': round(precision_score(y_test_bert_basic, preds_bert_basic), 2),
    'Recall test': round(recall_score(y_test_bert_basic, preds_bert_basic), 2),
    'Training time (s)': round(bert_basic_rs_search_time, 2)
}

# Добавляем строку к датафрейсу
results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

In [25]:
results

,Lemm method,F1 val,F1 test,Precision val,Precision test,Recall val,Recall test,Training time (s)
0,NLTK + TF-IDF + Logreg,0.63,0.64,0.51,0.52,0.84,0.85,224.33
1,SPACY + TF-IDF + Logreg,0.64,0.66,0.52,0.54,0.83,0.85,194.52
2,BERT_embedding + Logreg,0.65,0.65,0.51,0.51,0.88,0.89,358.74


#### Код эмбединга моделью берт

In [26]:
# import pandas as pd
# import numpy as np
# import torch
# import transformers
# from tqdm import tqdm

# Установка сидов
# SEED = 42
# torch.manual_seed(SEED)
# np.random.seed(SEED)

# Подключаем GPU, если есть
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# Загружаем модель и токенизатор
# MODEL_NAME = "bert-base-uncased"
# tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
# model = transformers.AutoModel.from_pretrained(MODEL_NAME)
# model.to(device)
# model.eval()

# Загружаем данные
# df = pd.read_csv("toxic_comments.csv")
# texts = df["text"].astype(str).tolist()

# Токенизация с обрезкой до 512
# print("Tokenizing...")
# encoded = tokenizer.batch_encode_plus(
#    texts,
#    add_special_tokens=True,
#    max_length=512,
#    truncation=True,
#    padding="longest",  # padding до самого длинного в батче
#    return_attention_mask=True,
#    return_tensors="pt"
# )

# input_ids = encoded["input_ids"]
# attention_mask = encoded["attention_mask"]

# Переносим на GPU
# input_ids = input_ids.to(device)
# attention_mask = attention_mask.to(device)

# Эмбеддинг
# batch_size = 32
# embeddings = []

# print("Generating embeddings...")
# with torch.no_grad():
#    for i in tqdm(range(0, input_ids.size(0), batch_size)):
#        input_batch = input_ids[i:i + batch_size]
#        mask_batch = attention_mask[i:i + batch_size]

#        outputs = model(input_batch, attention_mask=mask_batch)
#        cls_embeddings = outputs.last_hidden_state[0][:, 0, :]  # [CLS] токен
#        embeddings.append(cls_embeddings.cpu().numpy())

# Сохраняем в файл
# features = np.concatenate(embeddings, axis=0)
# np.save("bert_features.npy", features)

# print(f"Done! Embeddings shape: {features.shape}")

### BERT (после файнтюна)

In [27]:
# Предсказания на валидации и тесте
y_true_val_bert = bert_finetuned_prediction_val['true_label']
y_pred_val_bert = bert_finetuned_prediction_val['predicted_label']

y_true_test_bert = bert_finetuned_prediction_test['true_label']
y_pred_test_bert = bert_finetuned_prediction_test['predicted_label']


# Собираем метрики в строку
row = {
    'Lemm method': 'BERT_finetuning + BERT',
    'F1 val': round(f1_score(y_true_val_bert, y_pred_val_bert), 2),
    'Precision val': round(precision_score(y_true_val_bert, y_pred_val_bert), 2),
    'Recall val': round(recall_score(y_true_val_bert, y_pred_val_bert), 2),
    'F1 test': round(f1_score(y_true_test_bert, y_pred_test_bert), 2),
    'Precision test': round(precision_score(y_true_test_bert, y_pred_test_bert), 2),
    'Recall test': round(recall_score(y_true_test_bert, y_pred_test_bert), 2),
    'Training time (s)': round(16200, 2)
}

# Добавляем строку к датафрейсу
results = pd.concat([results, pd.DataFrame([row])], ignore_index=True)

In [28]:
results.sort_values(by='F1 val', ascending=False)

,Lemm method,F1 val,F1 test,Precision val,Precision test,Recall val,Recall test,Training time (s)
3,BERT_finetuning + BERT,0.83,0.82,0.86,0.86,0.79,0.78,16200.00
2,BERT_embedding + Logreg,0.65,0.65,0.51,0.51,0.88,0.89,358.74
1,SPACY + TF-IDF + Logreg,0.64,0.66,0.52,0.54,0.83,0.85,194.52
0,NLTK + TF-IDF + Logreg,0.63,0.64,0.51,0.52,0.84,0.85,224.33


Итого, для решения годится только вариант дообучения модели BERT результат которой удовлетворяет требованию Заказчика к значению метрики (F1 score) на уровне не менее 0.75.

#### Код файнтюна модели берт

In [29]:
#import pandas as pd
#import torch
#import time
#from datasets import Dataset
#from transformers import (
#    BertTokenizerFast,
#    BertForSequenceClassification,
#    TrainingArguments,
#    Trainer
#)
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

#start_total = time.time()

# 1. Загрузка данных
#start = time.time()
#df = pd.read_csv("/toxic_comments.csv")
#df["label"] = df["toxic"].astype(int)
#print(f"[1] Data loading time: {time.time() - start:.2f} sec")

# 2. Разделение на train / val / test
#start = time.time()
#df_trainval, df_test = train_test_split(
#    df, test_size=0.10, stratify=df["label"], random_state=42
#)
#df_train, df_val = train_test_split(
#    df_trainval, test_size=0.111, stratify=df_trainval["label"], random_state=42
#)
#print(f"[2] Data splitting time: {time.time() - start:.2f} sec")

# 3. Загрузка токенизатора
#start = time.time()
#tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
#print(f"[3] Tokenizer loading time: {time.time() - start:.2f} sec")

#def tokenize_function(examples):
#    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# 4. Преобразование в Dataset и токенизация
#start = time.time()
#train_dataset = Dataset.from_pandas(df_train).map(tokenize_function, batched=True)
#val_dataset = Dataset.from_pandas(df_val).map(tokenize_function, batched=True)
#test_dataset = Dataset.from_pandas(df_test).map(tokenize_function, batched=True)

#for ds in [train_dataset, val_dataset, test_dataset]:
#    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
#print(f"[4] Tokenization time: {time.time() - start:.2f} sec")

# 5. Загрузка модели
#start = time.time()
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
#print(f"[5] Model loading time: {time.time() - start:.2f} sec")

# 6. Метрики
#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()
#    preds = logits.argmax(axis=1)
#    return {
#        "accuracy": accuracy_score(labels, preds),
#        "precision": precision_score(labels, preds),
#        "recall": recall_score(labels, preds),
#        "f1": f1_score(labels, preds),
#        "roc_auc": roc_auc_score(labels, probs),
#    }

# 7. Аргументы обучения
#training_args = TrainingArguments(
#    output_dir="./bert_results",
#    evaluation_strategy="epoch",
#    save_strategy="epoch",
#    logging_strategy="epoch",
#    num_train_epochs=3,
#    per_device_train_batch_size=16,
#    per_device_eval_batch_size=32,
#    learning_rate=2e-5,
#    weight_decay=0.01,
#    load_best_model_at_end=True,
#    metric_for_best_model="f1",
#    fp16=True,
#    report_to="none",
#)

# 8. Тренировка
#start = time.time()
#trainer = Trainer(
#    model=model,
#    args=training_args,
#    train_dataset=train_dataset,
#    eval_dataset=val_dataset,
#    compute_metrics=compute_metrics,
#)
#trainer.train()
#print(f"[8] Training time: {time.time() - start:.2f} sec")

# 9. Финальная оценка на отложенном тесте
#start = time.time()
#test_results = trainer.evaluate(eval_dataset=test_dataset)
#print("\nFinal test metrics:", test_results)
#print(f"[9] Final evaluation time: {time.time() - start:.2f} sec")

# 10. Сохранение модели и токенизатора
#start = time.time()
#model.save_pretrained("./bert_model_finetuned")
#tokenizer.save_pretrained("./bert_model_finetuned")
#print(f"[10] Save time: {time.time() - start:.2f} sec")

#print(f"\nTotal time: {time.time() - start_total:.2f} sec")

#### Код инференса

In [30]:
#import time
#import torch
#import transformers
#import pandas as pd
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import f1_score, precision_score, recall_score
#from torch.utils.data import DataLoader, TensorDataset

# Пути к модели и данным
#model_path = "./арендаgpu/bert_finetune/model_for_sending_to_hf/"

# Загружаем токенайзер и модель
#tokenizer = transformers.BertTokenizer.from_pretrained(model_path)
#model = transformers.BertForSequenceClassification.from_pretrained(model_path)
#model.eval()

# Определяем устройство
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device)

# Загружаем данные
#df = pd.read_csv('./toxic_comments.csv')

# Разделение
#df_trainval_bert, df_test_bert = train_test_split(
#    df, test_size=0.10, stratify=df["toxic"], random_state=42
#)

#X_test_bert = df_test_bert['text'].tolist()
#y_test_bert = df_test_bert['toxic'].tolist()

# Токенизация
#inputs = tokenizer(
#    X_test_bert,
#    padding=True,
#    truncation=True,
#    max_length=512,
#    return_tensors="pt"
#)

# Создаем DataLoader для батчей (размер батча можно подобрать под память GPU)
#batch_size = 32
#dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
#dataloader = DataLoader(dataset, batch_size=batch_size)

#all_logits = []
#all_predictions = []
#start_time = time.time()

#with torch.no_grad():
#    for input_ids_batch, attention_mask_batch in dataloader:
#        input_ids_batch = input_ids_batch.to(device)
#        attention_mask_batch = attention_mask_batch.to(device)

#        outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch)
#        logits = outputs.logits

#        probs = torch.softmax(logits, dim=1)
#        preds = torch.argmax(probs, dim=1)

#        all_logits.append(logits.cpu())
#        all_predictions.extend(preds.cpu().tolist())

#elapsed_time = time.time() - start_time

# Объединяем все логииты в один тензор
#all_logits = torch.cat(all_logits, dim=0).numpy()

# Формируем DataFrame для выгрузки
#results_df = pd.DataFrame({
#    "text": X_test_bert,
#    "true_label": y_test_bert,
#    "predicted_label": all_predictions,
#})

# Добавляем колонки с логитами по классам
#for i in range(all_logits.shape[1]):
#    results_df[f"logit_class_{i}"] = all_logits[:, i]

# Сохраняем в CSV
#results_df.to_csv("bert_inference_results.csv", index=False)

# Метрики
#f1_test_bert = f1_score(y_test_bert, all_predictions, average='binary')
#precision_bert = precision_score(y_test_bert, all_predictions, average='binary')
#recall_bert = recall_score(y_test_bert, all_predictions, average='binary')

#print(f"F1 Score: {f1_test_bert:.4f}")
#print(f"Precision: {precision_bert:.4f}")
#print(f"Recall: {recall_bert:.4f}")
#print(f"Время выполнения предсказаний: {elapsed_time:.4f} секунд")
#print("Результаты сохранены в bert_inference_results.csv")

## Выводы

Рассмотрено три варианта решения задачи, удачным из которых оказался только один - файнтюнинг модели BERT. Требуемое заказчиком значение целевой метрики на уровне не ниже 0.75 достигнуто и равно 0.82, однако необходимо отметить ресурсоемкость данного решения для которого потребовалость 4.5 часа файнтюна на сервере со следующими характеристиками: gpu - 1xV100 32GB, CPU - 20 VCPU, RAM - 64Гб, SSD - 400Гб.

Все же складывается ощущуение, что решить задачу можно было намного проще... Сложность решения, в данном случае, скорее минус чем плюс.

## Итоговый вывод

В ходе проекта были рассмотрены три подхода к классификации токсичных комментариев:

- **Spacy + TF-IDF + LogisticRegression** — простая и быстрая, но недостаточно точная модель.
- **BERT (базовая модель) + LogisticRegression** — улучшение качества, но без дообучения ограничены возможности.
- **BERT (дообученный)** — достигнуто требуемое значение метрики F1 = **0.82**, что превышает порог 0.75.

Дообучение модели BERT оказалось наиболее эффективным решением, обеспечив качество, приемлемое для заказчика. Однако стоит учитывать высокие вычислительные затраты и время обучения (около 4.5 часов на мощном сервере).

Таким образом, баланс между качеством и ресурсами остаётся важным фактором при выборе подхода, и в текущем случае преимущества дообученной модели BERT перевешивают её недостатки.

---

**Рекомендация:** Для дальнейших проектов стоит рассмотреть упрощённые модели или оптимизацию процесса дообучения, чтобы снизить ресурсоёмкость без значительной потери качества.
